In [ ]:
import pandas as pd
df = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Entwicklungsdaten.tsv", sep="\t")
df1 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Trainingsdaten-GermEvalDatei2018.tsv", sep="\t", names=["c_text","hate","type","score"])
df2 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Trainingsdaten-GermEvalDatei2019.tsv", sep="\t", names=["c_text","hate","type","score"])
#df3 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Testdaten.tsv", sep="\t")

joined_frames = [df, df1, df2]
result = pd.concat(joined_frames)
text= result.iloc[0:, 1 ]

tempArray = []
for row in range(len(df1)):
    selected_row =df1.iloc[row, 2]
    if selected_row == "INSULT" or selected_row == "ABUSE":
        tempArray.append(1)
    else:
        tempArray.append(0)

df1['label'] = pd.Series(tempArray)

tempArray = []
for row in range(len(df2)):
    #print(df1.iloc[row, 2])
    selected_row =df2.iloc[row, 2]
    if selected_row == "INSULT" or selected_row == "ABUSE":
        tempArray.append(1)
    else:
        tempArray.append(0)

df2['label'] = pd.Series(tempArray)

hs = (df.iloc[0: , 8 ].to_list() + df1.iloc[0: , 4].to_list() + df2.iloc[0: , 4].to_list())

In [ ]:
import spacy
nlp = spacy.load("de_core_news_sm")

array =[]
for t in text[:]:
    doc = nlp(t)
    #print([token.text for token in doc])
    
    #normalize dataset
    
    #with stop words
    #temp = [token.lemma_.lower() for token in doc if token.is_alpha] #36
    
    #without stop words and only alpha
    #temp = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
    
    #without stop words and with smileys /no alpha chars
    temp = [token.lemma_.lower() for token in doc if not token.is_stop] 
    
    #with smileys and stopwords
    #temp = [token.lemma_.lower() for token in doc]
    
    #print(temp)

    array.append((" ".join(temp)))

#print (array)

df = pd.DataFrame ({'Text': array , 'Hatespeech': hs })
#print (df)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.Text, 
    df.Hatespeech, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2021,#-> same order of dataset random_state=2022, 2021
    stratify=df.Hatespeech
)

#As this dataset is highly imbalance we have to balance this by over sampling

trainingdata = {
    "string": X_train,
    "label": y_train
}
newDf = pd.DataFrame(trainingdata)

print(newDf)

cnt_non_fraud = newDf[newDf['label'] == 0]['string'].count()
df_class_fraud = newDf[newDf['label'] == 1]
df_class_nonfraud = newDf[newDf['label'] == 0]
df_class_fraud_oversample = df_class_fraud.sample(cnt_non_fraud, replace=True)
df_oversampled = pd.concat([df_class_nonfraud, df_class_fraud_oversample], axis=0)

print('Random over-sampling:')
print(df_oversampled['label'].value_counts())
print(df_oversampled)

X_train = df_oversampled['string']
y_train = df_oversampled['label']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification 
import xgboost as xgb


#vecktorisieren 
vectorizer = TfidfVectorizer() 
Xtrain = vectorizer.fit_transform(X_train) 
Xtest = vectorizer.transform(X_test) 
print(Xtrain.shape)

#create classifier and train model 
clf = xgb.XGBClassifier(objective="binary:logistic", random_state=2021,# tree_method="gpu_hist", enable_categorical=True,  #booster='gblinear', 2021

                        n_estimators = 1000,
                        gamma=0.63, #0.61, #2.04
                        min_child_weight=0.1,
                        reg_alpha=0,
                        max_depth=14, #14 27
                        colsample_bytree=1, colsample_bylevel=0.5, colsample_bynode=1
                         )
                        

clf.fit(Xtrain, y_train)

#3. get the predictions for X_test and store it in y_pred 
y_pred = clf.predict(Xtest)

#4. print the classfication report 
print(classification_report(y_test, y_pred))

#find accuracy scores
accuracy = accuracy_score(y_test, y_pred) 
print("The accuracy of prediction is: ", accuracy)

#6. count prediction targets
hs=1
nhs=0

for p in y_pred:
    if p == 1:
        hs+=1
    if p == 0:
        nhs+=1
print("hs: ", hs)
print("nhs: ", nhs)

In [ ]:
while True:
    choice = input("Enter Choice: ")
    #print(choice)
    
    array =[]
    doc = nlp(choice)

        #without stop words and with smileys /no alpha chars
    temp = [token.lemma_.lower() for token in doc if not token.is_stop] # 0.45 f1

    array.append((" ".join(temp)))
    
    Xtest = vectorizer.transform(array) 
    y_pred = clf.predict(Xtest)
    print(y_pred)
    